# IBM Data Science Capstone

<img src="https://i.imgflip.com/2xj7o2.jpg" alt="drawing" width="200"/>

Hello there! 

Thanks for stopping by. Been a tough ride this specialization, hasn't it? All of the typos, the random quizzes, the annoying peer-reviews, and the not-so-helpful instructors. Well, we're almost done, so yay!

The capstone should be fun. I hope. 
<img src="https://i.imgflip.com/qnmnz.jpg" alt="drawing" width="200"/>

In [2]:
#import numpy as npy # let's not get carried away now
import numpy as np
import pandas as pd

In [3]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!
